# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Aluno: Marcos Carvalho Ferreira

In [1]:
%pip install pulp
import pulp

Note: you may need to restart the kernel to use updated packages.


## Problema 2

Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é de R\$ 2,00 nas duas primeiras semanas e R\$ 2,50 nas duas últimas, quando a concorrência demandar por material no mercado. A demanda semanal de camisetas será de 5.000, 10.000, 30.000 e 60.000. A capacidade máxima de produção da empresa é de 25.000 camisetas semanalmente. Na primeira e na segunda semanas, a empresa poderá contratar horas extras de serviço e fabricar mais 10.000 camisetas em cada semana. Nesse caso, o custo de produção sobe para R\$ 2,80. O excesso de produção pode ser estocado a um custo de R\$ 0,20 por unidade por semana. Formule um modelo que minimize os custos.

## Parâmetros

Dados (do enunciado):

* Demanda semanal $d = [5000,\,10000,\,30000,\,60000]$.
* Capacidade normal por semana: $25\,000$.
* Capacidade de hora extra (semanas 1 e 2): $10\,000$ cada.
* Custo de produção normal: $c_t = 2{,}00$ nas semanas 1 e 2; $c_t = 2{,}50$ nas semanas 3 e 4.
* Custo de hora extra: $c^{HE}=2{,}80$.
* Custo de estocagem por unidade por semana: $h = 0{,}20$.
* Estoque inicial $E_0 = 0$.

## Variáveis de decisão

Para cada semana $t=1,2,3,4$:

* $p_t$ = quantidade produzida **em regime normal** na semana $t$ (unidades).
* $he_t$ = quantidade produzida em **hora extra** na semana $t$ (unidades).
  Observação: $he_3=he_4=0$ (horas extras só são permitidas nas semanas 1 e 2).
* $E_t$ = estoque (inventário) ao **final** da semana $t$ (unidades).

In [2]:
semanas = [1, 2, 3, 4]

demanda = {
    1: 5000,
    2: 10000,
    3: 30000,
    4: 60000
}

custo_normal = {
    1: 2.0,
    2: 2.0,
    3: 2.5,
    4: 2.5
}

custo_extra = 2.8
custo_estoque = 0.2

capacidade_normal = {
    1: 25000,
    2: 25000,
    3: 25000,
    4: 25000
}

capacidade_extra = {
    1: 10000,
    2: 10000,
    3: 0,
    4: 0
}

## Modelo matemático

### Função Objetivo:

Minimize os custos: produção, produção em hora extra e estoque.

$$
\min \; f = \sum_{t=1}^{4} c_t\,p_t \;+\; c^{HE}\sum_{t=1}^{2} he_t \;+\; h\sum_{t=1}^{4} E_t
$$

### Restrições:

1. Equilíbrio de estoque (restrições de balanço) para cada semana:

$$
E_t = E_{t-1} + p_t + he_t - d_t,\qquad t=1,2,3,4,
$$

com condição inicial: $E_0=0$.

2. A produção normal deve respeitar o limite de capacidade normal

$$
p_t \le 25\,000 \quad\forall t.
$$

3. A produção em hora extra deve respeitar o limite de capacidade em hora extra. Lembrando que a empresa não pode contratar horas extras nas semanas 3 e 4:

$$
0 \le he_t \le 10\,000\quad (t=1,2),\qquad he_3=he_4=0 .
$$

4. Domínio:

$$
p_t \ge 0,\quad he_t \ge 0,\quad E_t \ge 0 \quad\forall t.
$$

### Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem("ProducaoCamisetas", pulp.LpMinimize)

### Variáveis de Decisão

In [4]:
p = pulp.LpVariable.dicts("ProducaoNormal", semanas, lowBound=0)
he = pulp.LpVariable.dicts("ProducaoExtra", semanas, lowBound=0)
E = pulp.LpVariable.dicts("Estoque", semanas, lowBound=0)

### Função Objetivo

In [5]:
modelo += (
    pulp.lpSum(custo_normal[t] * p[t] for t in semanas) +
    pulp.lpSum(custo_extra * he[t] for t in semanas) +
    pulp.lpSum(custo_estoque * E[t] for t in semanas)
)

### Restrições

In [6]:
for t in semanas:
    modelo += p[t] <= capacidade_normal[t], f"CapacidadeNormal_{t}"
    modelo += he[t] <= capacidade_extra[t], f"CapacidadeExtra_{t}"

E_anterior = 0
for t in semanas:
    modelo += E[t] == E_anterior + p[t] + he[t] - demanda[t], f"Balanço_{t}"
    E_anterior = E[t]

In [7]:
modelo

ProducaoCamisetas:
MINIMIZE
0.2*Estoque_1 + 0.2*Estoque_2 + 0.2*Estoque_3 + 0.2*Estoque_4 + 2.8*ProducaoExtra_1 + 2.8*ProducaoExtra_2 + 2.8*ProducaoExtra_3 + 2.8*ProducaoExtra_4 + 2.0*ProducaoNormal_1 + 2.0*ProducaoNormal_2 + 2.5*ProducaoNormal_3 + 2.5*ProducaoNormal_4 + 0.0
SUBJECT TO
CapacidadeNormal_1: ProducaoNormal_1 <= 25000

CapacidadeExtra_1: ProducaoExtra_1 <= 10000

CapacidadeNormal_2: ProducaoNormal_2 <= 25000

CapacidadeExtra_2: ProducaoExtra_2 <= 10000

CapacidadeNormal_3: ProducaoNormal_3 <= 25000

CapacidadeExtra_3: ProducaoExtra_3 <= 0

CapacidadeNormal_4: ProducaoNormal_4 <= 25000

CapacidadeExtra_4: ProducaoExtra_4 <= 0

Balanço_1: Estoque_1 - ProducaoExtra_1 - ProducaoNormal_1 = -5000

Balanço_2: - Estoque_1 + Estoque_2 - ProducaoExtra_2 - ProducaoNormal_2
 = -10000

Balanço_3: - Estoque_2 + Estoque_3 - ProducaoExtra_3 - ProducaoNormal_3
 = -30000

Balanço_4: - Estoque_3 + Estoque_4 - ProducaoExtra_4 - ProducaoNormal_4
 = -60000

VARIABLES
Estoque_1 Continuous
Estoqu

### Resolvendo o problema

In [8]:
modelo.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/marcos/Área de trabalho/Mestrado/1 - Disciplina Isolada/Programação Matemática/Atividades Práticas/Atividade 1/atividade1/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/0863dbee3cbb4d7182ffd07447a74160-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/0863dbee3cbb4d7182ffd07447a74160-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 53 RHS
At line 66 BOUNDS
At line 67 ENDATA
Problem MODEL has 12 rows, 12 columns and 23 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-11) rows, 7 (-5) columns and 7 (-16) elements
0  Obj 114999.13 Primal inf 65000.405 (1)
1  Obj 258000
Optimal - objective value 258000
After Postsolve, objective 258000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 258000 - 1 iterations time 0.002, Pres

1

## Imprimindo as soluções do problema

In [9]:
print("Status:", pulp.LpStatus[modelo.status])
print("Custo total:", pulp.value(modelo.objective))

for t in semanas:
    print(f"Semana {t}: Produção normal={p[t].value():.0f}, "
          f"Extra={he[t].value():.0f}, Estoque final={E[t].value():.0f}")

Status: Optimal
Custo total: 258000.0
Semana 1: Produção normal=25000, Extra=0, Estoque final=20000
Semana 2: Produção normal=25000, Extra=5000, Estoque final=40000
Semana 3: Produção normal=25000, Extra=0, Estoque final=35000
Semana 4: Produção normal=25000, Extra=0, Estoque final=0
